In [1]:
# Heat budget for the different
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [12]:
Xp1 = 51 # Number of cell corner points
# Find the uppler and lower layer depth above the sill crest
def Layer(u):
    # Layer interface, zero velocity crossing point
    z_th = np.empty(Xp1)
    itf = np.empty(Xp1, dtype=int)
    z_th[[0,1,-2,-1]] = 0
    itf[[0,1,-2,-1]] = 0
    for i in range(2,Xp1-2):
        l = np.where(u[:, i] < 0)[-1][0]
        itf[i] = l
        z_th[i] = -w[l, i] * (pt[l-1:l+1,i].mean()-T0)
    return (itf, z_th)

In [27]:
Cp = 4220  # Specific heat content J/kg/C
rho = 1025 # Density
T0 = 9 # Initial temperature
ny = 9 # Number of grids in Y direction
ga = np.repeat(np.array([[.5],[1],[1.5],[4],[8]]),10) * 200 # Grid area


files = sorted(glob.glob('/Users/weiyangbao/Documents/Model_outputs/State700/Hs*.nc'))
Hs = np.array([10, 12, 15, 110])

H_f = np.empty(len(Hs)) # Heat flux through the ice front
H_f1 = np.empty(len(Hs)) # 1: upper layer
H_s = np.empty(len(Hs)) # Heat flux through the sill
H_s1 = np.empty(len(Hs))
H_z = np.empty(len(Hs)) # Vertical heat flux
for i in range(len(Hs)):
    state0 = xr.open_dataset(files[i])
    t0 = 0 # Start from day 20
    tn = len(state0.T) # End day
    state = state0.isel(Y = range(1,10), T = range(t0,tn)) # Remove the boundary walls in y direction
    pt = state.Temp.data.mean(axis=(0,2)) # Z, X
    w = state.W.data.mean(axis=(0,2)) # Zl, X
    u = state.U.data.mean(axis=(0,2)) 
    itf, z_th = Layer(u)
    # sill crest: itf[27]
    H_z[i] = z_th[2:29].sum() * 40000 * ny
    H_f[i] = np.sum(u[:,2] * (pt[:,1]-T0) * ga) * ny
    H_f1[i] = np.sum(u[:itf[2],2] * (pt[:itf[2],1]-T0) * ga[:itf[2]]) * ny
    H_s[i] = -np.sum(u[:,28] * (pt[:,27]-T0) * ga) * ny
    H_s1[i] = -np.sum(u[:itf[28],28] * (pt[:itf[28],27:29].mean(1)-T0) * ga[:itf[28]]) * ny
itf[28]

16

    H_z[i] = z_th[2:29].sum() * 40000 * ny
    Qf = u[:,2] * (pt[:,1]-T0) * Area * ny
    H_f[i] = Qf.sum()
    #H_f2[i] = Qf[np.int(itf[2]):].sum()
    H_f2[i] = Qf[Qf>0].sum()
    Qs = -u[:,28] * (pt[:,27]-T0) * Area * ny
    H_s[i] = Qs.sum()
    H_s2[i] = Qs[np.int(itf[28]):].sum()

In [28]:
H_z,H_f, H_s, H_f+H_s

(array([-4344.30991727, -2736.43736357,  -709.57352969,   408.75530743]),
 array([-5487.81237323, -5561.86994947, -5668.96521575, -5631.00153356]),
 array([3527.43614763, 3920.319888  , 4792.94689894, 6290.99725052]),
 array([-1960.3762256 , -1641.55006148,  -876.01831681,   659.99571696]))

In [ ]:
#fig, ax =plt.subplots(figsize=(10,6))
fig, (ax1, ax2) = plt.subplots(1,2,sharex=True, figsize=(16,16))
labels = [10, 12, 15]
#ax1.set_xticklabels(labels)
ax1.grid(linestyle='--')
plt.yticks(fontsize = 15)
plt.xticks(np.arange(3), labels, fontsize=15)
plt.xlabel('$H_s\;(m)$', size=20)
plt.ylabel('Heat flux ($\circ C\cdot m^3/s$)', size=20)
plt.title('Hf', size=20)
ax1.plot(np.arange(3), H_f, 'o-', lw=2)

ax2.grid(linestyle='--')
plt.yticks(fontsize = 15)
plt.xticks(np.arange(3), labels, fontsize=15)
plt.xlabel('$H_s\;(m)$', size=20)
#ax2.ylabel('Heat flux ($\circ C\cdot m^3/s$)', size=20)
plt.title('Hs', size=20)
ax2.plot(np.arange(3), H_s, 'o-', lw=2)

In [ ]:
files